In [1]:
import numpy as np
from sklearn.svm import SVR, SVC, LinearSVR, LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from data_generation import get_data, g_0, m_0
from ml_models import get_model_m, get_model_g
from dml_algorithm import dml_ate

In [2]:
rng = np.random.default_rng(seed=42)

In [65]:
N = 2000
y_data, d_data, x_data = get_data(N, rng)
y_train, y_test, d_train, d_test, x_train, x_test = train_test_split(y_data, d_data, x_data, test_size=0.2, random_state=42)

In [66]:
model_g0, model_g1 = LinearSVR(dual='auto', max_iter=2500), LinearSVR(dual='auto', max_iter=2500)
model_m = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', max_iter=2500)) #, n_jobs=-1
model_m_nonlinear = SVC(probability=True, random_state=42)

In [67]:
model_m_nonlinear.fit(x_train, d_train)
print(model_m_nonlinear.predict_proba(x_test)[:20,1])

[0.60485627 0.29968854 0.52736136 0.25401708 0.64742263 0.3694982
 0.14884836 0.4843868  0.67344888 0.26837934 0.53853743 0.10862596
 0.44890228 0.4467778  0.30750706 0.76262885 0.42810266 0.36593837
 0.1634676  0.46501755]


In [68]:
%%time
model_m.fit(x_train, d_train)
print(model_m.predict_proba(x_test)[:20,1])
print(m_0(x_test[:20]))

[0.77865401 0.28529552 0.52399473 0.21187097 0.71483329 0.36747679
 0.21275911 0.44823947 0.84845244 0.31918226 0.61958862 0.09412965
 0.56548691 0.6232501  0.40453398 0.704828   0.32395574 0.38388663
 0.16141041 0.5626135 ]
[0.83187818 0.25823867 0.58951401 0.20381279 0.72388698 0.39303329
 0.20888606 0.5086755  0.8978375  0.39509225 0.6086003  0.0954005
 0.58603231 0.61484161 0.38291112 0.76633555 0.34977621 0.45467357
 0.21964476 0.57454044]
CPU times: total: 62.5 ms
Wall time: 73.7 ms


In [7]:
%%time
model_g0.fit(x_train[d_train==0], y_train[d_train==0])
print(model_g0.predict(x_test[:20]))
print(g_0(0, x_test[:20]))

[-0.78173141  0.30313703  2.90054586  1.13767755  1.98670908  4.22057655
  6.83429797  8.70135277  3.63499673  7.47601521  4.52370501  2.10789304
  3.61239182  5.06141432  2.59417429  1.48209818  5.92067223  2.56904601
  1.43833343  3.16899719]
[-0.37587051  2.69627016  2.02086517  2.79051761  1.99831557  3.49612863
  6.32549494 10.61562091  3.37291078  9.15138507  3.96434201  2.03191128
  3.09661065  4.74037632  2.30164594  2.38750621  5.86916474  1.98684077
  1.18209388  4.99912948]
CPU times: total: 62.5 ms
Wall time: 115 ms


In [8]:
%%time
model_g1.fit(x_train[d_train==1], y_train[d_train==1])
print(model_g1.predict(x_test[:20]))
print(g_0(1, x_test[:20]))

[0.79131711 0.32978026 2.04131291 2.19077042 2.52778668 3.53044148
 7.11595151 8.50334853 5.89062787 7.10953625 4.64543103 3.23967675
 3.83653026 5.85732953 3.27153862 3.7852954  3.99316109 2.1443544
 1.12828251 3.48715826]
[0.90193897 2.91504961 1.19098654 3.69058595 2.74056392 2.60898902
 6.4891349  9.80995146 6.44321947 8.38270697 4.04521336 3.08347733
 3.11994487 5.42606548 2.82474951 5.08378817 4.5084311  1.92677582
 1.02831776 5.68516164]
CPU times: total: 78.1 ms
Wall time: 92.9 ms


In [ ]:
C, max_iter

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100]
}

In [255]:
%%time
N = 16000
y_data, d_data, x_data = get_data(N, rng)
model_g0, model_g1 = LinearSVR(C=5, dual='auto', max_iter=5000, random_state=42), LinearSVR(C=5, dual='auto', max_iter=5000, random_state=42)
model_g = [model_g0, model_g1]
model_m = CalibratedClassifierCV(estimator=LinearSVC(C=5, dual='auto', max_iter=5000, random_state=42)) #, n_jobs=-1
dml_ate(y_data, d_data, x_data, model_g, model_m, K=5, classical=True, inference=True, alpha=0.05)

CPU times: total: 3.02 s
Wall time: 3 s


(array([0.51908536, 0.55552273, 0.45258778]),
 5.451014995427961,
 array([0.43462256, 0.60354816]))

In [46]:
%%time
model_g0, model_g1 = LinearSVR(dual='auto', max_iter=2500, random_state=42), LinearSVR(dual='auto', max_iter=2500, random_state=42)
model_g = [model_g0, model_g1]
model_m = SVC(probability=True, random_state=42)
dml_ate(y_data, d_data, x_data, model_g, model_m, K=5, classical=True, inference=True, alpha=0.05)

CPU times: total: 984 ms
Wall time: 986 ms


(array([0.5423967 , 0.64183653, 0.10269771]),
 4.224612415082437,
 array([0.28055733, 0.80423608]))

In [193]:
def svm_cv(y_data, d_data, x_data, cv=5):
    model_g = LinearSVR(dual='auto', max_iter=5000, random_state=42)
    #model_m = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', max_iter=5000, random_state=42))
    model_m = SVC(kernel='linear', probability=True, random_state=42)
    
    param_grid = {
        'C': [0.1, 1, 3, 10, 5]
    }

    grid_search_g = GridSearchCV(estimator=model_g, param_grid=param_grid, cv=cv, n_jobs=-1,
                                 scoring='neg_mean_squared_error')
    grid_search_m = GridSearchCV(estimator=model_m, param_grid=param_grid, cv=cv, n_jobs=-1,
                                 scoring='neg_brier_score')

    svm_params_dict = {}
    for d in [0, 1]:
        grid_search_g.fit(X=x_data[d_data == d], y=y_data[d_data == d])
        svm_params_dict[f'g{d}'] = grid_search_g.best_params_
    grid_search_m.fit(X=x_data, y=d_data)
    svm_params_dict['m'] = grid_search_m.best_params_

    return svm_params_dict

In [198]:
for N in [250, 500, 1000]:#, 2000, 4000, 8000, 16000]:
    y_data, d_data, x_data = get_data(N, rng)
    svm_params_dict = svm_cv(y_data, d_data, x_data)
    print(svm_params_dict)

{'g0': {'C': 10}, 'g1': {'C': 5}, 'm': {'C': 10}}
{'g0': {'C': 5}, 'g1': {'C': 3}, 'm': {'C': 10}}


C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'g0': {'C': 10}, 'g1': {'C': 10}, 'm': {'C': 1}}


In [ ]:
choose C=5!